In [3]:
from langchain.agents import initialize_agent, Tool
from langchain_upstage import ChatUpstage
from langchain.memory import ConversationBufferMemory
from langchain.agents.agent_types import AgentType
import os
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
import os

from langchain_upstage import ChatUpstage
from langgraph.graph import StateGraph, END
from typing import TypedDict

# 1. 상태 정의
class AgentState(TypedDict):
    history: list[str]
    turn: int

# 2. 두 에이전트 정의
llm_a = ChatUpstage(api_key=os.getenv("SOLAR_API_KEY"), model='solar-pro', max_tokens=100)
llm_b = ChatUpstage(api_key=os.getenv("SOLAR_API_KEY"), model='solar-pro', max_tokens=100)

def agent_a_node(state: AgentState) -> AgentState:
    msg = state["history"][-1]
    response = llm_a.invoke(f"[역할: 아이디어 제안자] {msg}")
    return {"history": state["history"] + [f"A: {response}"], "turn": state["turn"] + 1}

def agent_b_node(state: AgentState) -> AgentState:
    msg = state["history"][-1]
    response = llm_b.invoke(f"[역할: 비판적 검토자] {msg}")
    return {"history": state["history"] + [f"B: {response}"], "turn": state["turn"] + 1}

def check_done(state: AgentState) -> str:
    last_msg = state["history"][-1]
    if "결론" in last_msg or state["turn"] >= 6:
        return END
    return "a" if state["turn"] % 2 == 0 else "b"

# 3. 그래프 정의
workflow = StateGraph(AgentState)
workflow.add_node("a", agent_a_node)
workflow.add_node("b", agent_b_node)
workflow.set_entry_point("a")
workflow.add_conditional_edges("a", check_done)
workflow.add_conditional_edges("b", check_done)

graph = workflow.compile()

# 4. 실행
initial_state = {"history": ["살 안찌고 맛있는 저녁 메뉴 추천해줘."], "turn": 0}
final_state = graph.invoke(initial_state)

# 결과 출력
print("\n\n🧠 전체 대화 로그:")
for msg in final_state["history"]:
    print(msg)




🧠 전체 대화 로그:
지구 온난화를 해결할 수 있는 방안을 제시해줘.
A: content='지구 온난화를 해결하기 위한 몇 가지 아이디어를 제시해 드리겠습니다:\n\n1. 재생 에너지로의 전환: 화석 연료를 태양, 풍력, 수력과 같은 재생 에너지원으로 대체함으로써 온실 가스 배출을 크게 줄일 수 있습니다.\n\n2. 에너지 효율성 향상: 에너지 효율적인 가전제품, 건물 및 교통 수단을 사용하여 에너지 소비를 줄일 수 있습니다.\n\n3. 재조림 및 조림: 나무를 더 많이 심고 숲을 복원함으로써 대기 중의 이산화탄소를 흡수하고 저장할 수 있습니다.\n\n4. 지속 가능한 농업: 지속 가능한 농업 관행을 통해 토양 건강과 생물 다양성을 개선하고 메탄 배출을 줄일 수 있습니다.\n\n5. 폐기물 감소 및 재활용: 폐기물 발생을 줄이고 재활용을 늘림으로써 매립지에서 발생하는 메탄 배출을 줄일 수 있습니다.\n\n6. 대중 교통 및 친환경 교통 수단 장려: 대중 교통, 자전거, 도보 및 전기 자동차와 같은 친환경 교통 수단을 장려함으로써 온실 가스 배출을 줄일 수 있습니다.\n\n7. 탄소 포집 및 저장: 발전소, 산업 공정 및 기타 소스에서 배출되는 이산화탄소를 포집하여 지하에 저장함으로써 대기 중에서 제거할 수 있습니다.\n\n8. 교육 및 인식 제고: 대중에게 지구 온난화의 원인과 결과에 대해 교육하고, 개인이 탄소 발자국을 줄이기 위해 취할 수 있는 조치에 대해 알리는 것이 중요합니다.\n\n이러한 아이디어들은 함께 실행될 때 지구 온난화를 해결하고 그 영향을 완화하는 데 도움이 될 수 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 323, 'prompt_tokens': 29, 'total_tokens': 352, 'completion_tokens_details': None, 'prompt_tokens_details': None}